In [6]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

  Using cached transformers-3.0.2-py3-none-any.whl (769 kB)
  Using cached tokenizers-0.8.1rc1-cp37-cp37m-manylinux1_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.12.1
    Uninstalling tokenizers-0.12.1:
      Successfully uninstalled tokenizers-0.12.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.18.0
    Uninstalling transformers-4.18.0:
      Successfully uninstalled transformers-4.18.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kobert 0.2.3 requires transformers>=4.8.1, but you have transformers 3.0.2 which is incompatible.


In [2]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-jsm9ez1s
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-jsm9ez1s
     |████████████████████████████████| 132 kB 4.3 MB/s 
     |████████████████████████████████| 4.5 MB 19.3 MB/s 
     |████████████████████████████████| 4.0 MB 37.3 MB/s 
     |████████████████████████████████| 596 kB 46.9 MB/s 
     |████████████████████████████████| 77 kB 6.4 MB/s 
     |████████████████████████████████| 6.6 MB 34.7 MB/s 
     |████████████████████████████████| 8.7 MB 19.9 MB/s 
     |████████████████████████████████| 79 kB 6.9 MB/s 
     |████████████████████████████████| 127 kB 43.4 MB/s 
  Created wheel for kobert: filename=kobert-0.2.3-py3-none-any.whl size=15674 sha256=bfaebf65e70cc0084a1dc7c77cc4a3237d35acda8e25a70d436b43c07a376a55
  Stored in directory: /tmp/pip-ephem-wheel-cache-d1lrebsx/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd05

In [7]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [8]:
import pandas as pd
data = pd.read_csv('total_final.csv')

In [9]:
data.head()

,Unnamed: 0,type,review,star,label
0,0,금성침대 파인드 LED 서랍형 침대 SS,좋은 가격에 잘 구매했습니다 다만 수납공간은 조금 많이 작네요 이 정도였으면 그냥 ...,3,0
1,1,금성침대 파인드 LED 서랍형 침대 SS,침대 프레임이 정말 실망스럽네요 기존의 싼 제품과 비교까지 하는 상품설명을 보고 구...,1,0
2,2,금성침대 파인드 LED 서랍형 침대 SS,별로예요 사진처럼 침대 하단부에 시트지가 조금 벗겨졌어요 기사님 설치하실 때 그런 ...,2,0
3,3,금성침대 파인드 LED 서랍형 침대 SS,설치다 하시고 기사님은 가셨고 프레임 헤드 쪽이 깨져 있었어요 못 보신 건지 말씀 ...,2,0
4,4,금성침대 파인드 LED 서랍형 침대 SS,월일에 설치해서 딱 두 달 사용했는데 서랍장이 분리되어 버렸습니다 서랍장이 톱밥 뭉...,1,0


In [10]:
data = data.drop(['Unnamed: 0'], axis=1)
data = data.drop(['star'], axis=1)
data = data.drop(['type'], axis=1)

In [11]:
data

,review,label
0,좋은 가격에 잘 구매했습니다 다만 수납공간은 조금 많이 작네요 이 정도였으면 그냥 ...,0
1,침대 프레임이 정말 실망스럽네요 기존의 싼 제품과 비교까지 하는 상품설명을 보고 구...,0
2,별로예요 사진처럼 침대 하단부에 시트지가 조금 벗겨졌어요 기사님 설치하실 때 그런 ...,0
3,설치다 하시고 기사님은 가셨고 프레임 헤드 쪽이 깨져 있었어요 못 보신 건지 말씀 ...,0
4,월일에 설치해서 딱 두 달 사용했는데 서랍장이 분리되어 버렸습니다 서랍장이 톱밥 뭉...,0
...,...,...
21304,폭이 아쉽기는 하지만 튼튼하고 분위기 있어요,1
21305,잘 사용하고 있어요 좋아요,1
21306,튼튼하고 수납공간도 좋아서 여전히 만족해요,1
21307,지방이다 보니 배송이 좀 늦고 기사님이 바쁘셔서 그냥 집에 두고 가시다 보니 균형 ...,1


In [12]:
data_list = []
for q, label in zip(data['review'], data['label'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [13]:
print(data_list[0])
print(data_list[6000])
print(data_list[12000])
print(data_list[18000])
print(data_list[-1])

['좋은 가격에 잘 구매했습니다 다만 수납공간은 조금 많이 작네요 이 정도였으면 그냥 없는 모델 살 걸 그랬네요 라이트도 불이 단계 조절이 되는 걸로 알고 있었는데 어떻게 하는지 설명서가 없어서 상품 페이지 보고 알았네요 매트리스는 진짜 끝내주네 요 매트는 별 다섯입니다 배송과 설치는 빨리 잘해주셨는데 기다린 시간이 너무 오래 걸리네 요 설치는 잘 받았어요', '0']
['한 달 써본 결과 좋긴 한데 쿠션감은 좀 없어요', '1']
['배송일보다 좀 일찍 도착했습니다 상태 좋아요', '1']
['상품은 괜찮은데 배송료가 지방이라 너무 비쌈 물건은 원인가 하는데 배송료 원 배송은 온다 안 온다 온다 안 온다 몇 번을 거듭하고 진짜 짜증 나서 할 수 있으면 취소하고 싶었음 해외 배송도 아닌데 한 달 걸림년 인생 중 제일 열받은 물건 ㅡㅡ', '0']
['앉아서 오래 일해서 허리가 아파서 구매했습니다 설치하고 시간 정도 정리한 거 같아 요 너무너무 맘에 들어 요 사이즈인데 키보드 칠 때마다 약간씩 모니터 흔들거림이 있네요 그래도 튼튼한 거 같아', '1']


In [17]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split                                            
dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, random_state=0)

In [18]:
print(len(dataset_train))
print(len(dataset_test))

17047
4262


In [19]:
dataset_train

[['만족해요', '1'],
 ['침대에 전기 콘센트가 있어서 딸이 좋아합니다 그레이 색상도 맘에 들어 하고요', '0'],
 ['색상 디자인 너무 깔끔하네요', '1'],
 ['이사 후에 몬스 침대방과도 너무 잘 어울리고 너무나 편안한 잠자리로 매일 밤 수면제로 청했던 잠을 수면제도 안 먹고 꿀잠을 잔답니다 이웃과 친구에게도 강력 추천하고 있답니다',
  '1'],
 ['가격 대비 이만한 제품은 없으나 자세히 보면 마감이 별로예요', '0'],
 ['좋습니다 디자인 색상 다 맘에 들어 요 마감 처리는 좀 부족한 부분도 있긴 하지만 전체적으로 좋습니다', '1'],
 ['디자인 심플하고 견고하네요', '1'],
 ['사이즈 랑 디자인은 딱 그대로입니다만 어찌나 합판 냄새가 구린 기아주 그냥 후회가 될 정도로 그냥 그래요 이 더위에 환기를 해도 저녁엔 ㅠㅠ',
  '0'],
 ['옷걸이가 생각보다 높이 있는 거 빼고는 다 맘에 들어요', '1'],
 ['아들 방에 설치했어요 하나 더 구매해야겠어요', '0'],
 ['이거 절대 사지 마세요 오다 자인은 개안아요 그런 게 옷 좀 걸면 봉이 무너집니다 불안해서 옷을 못 거네요 내일 고객센터 전화해 바서 근본적으로 해결안 대면 고발하고 불매운동이라도 할 겁니다 아니 옷걸이가 옷을 못 걸면 어쩌라 구이 따위로 만드는지',
  '0'],
 ['한 달 써보니 견고하고 수납공간도 충분합니다', '1'],
 ['프레임은 만족합니다 매트리스는 돈을 더 주더라도 다른 걸로 할걸 그랬어요 누워 있는데 움직이면 스프링 튕기는 소리가 나요ㅠㅠ',
  '1'],
 ['주문 후 제품 도착까지 일이 소요되었습니다', '0'],
 ['최고예요 이만한 가격에 설치까지 해주시 다니 정말 구성비 최고에 요 직접 사서 조립하는 제품도 가격이 만만치 않더군요 그래서 선택했는데 기사님도 정말 친절하셨고 마무리도 꼼꼼하셨습니다 집 인테리어와 도 정말 잘 어울려 요 이사한 뒤로 옷 걸을 데가 없었는데 생기자마자 바로 정리했습니다ㅎㅎ',
  '1'],
 ['

In [20]:
device = torch.device("cuda:0")

In [21]:
#bert 모델, vocab 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [22]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [23]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [24]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [25]:
#BERTDataset 클래스 이용, TensorDataset으로 만들어주기
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [26]:
data_train[0]

(array([   2, 1947, 7848, 6999,    3,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(5, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 1)

In [27]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [28]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [29]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [30]:
#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning


In [31]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/267 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 0.7366468906402588 train acc 0.5
epoch 1 batch id 201 loss 0.5466116070747375 train acc 0.7135416666666666
epoch 1 train acc 0.7348024548119199


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 4 test acc 0.8318317949725059


  0%|          | 0/267 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.2772311866283417 train acc 0.890625
epoch 5 batch id 201 loss 0.1779962182044983 train acc 0.9231965174129353
epoch 5 train acc 0.9261012050154698


  0%|          | 0/67 [00:00<?, ?it/s]

epoch 5 test acc 0.8354894933228595


In [32]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("부정")
            elif np.argmax(logits) == 1:
                test_eval.append("긍정")

        print(">> 입력한 리뷰는 " + test_eval[0] + " 입니다.")

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
# 무한반복 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("리뷰데이터 : ")
    if sentence == 0 :
        break
    predict(sentence)
    print("\n")

리뷰데이터 : 예쁘기만 함


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


>> 입력한 리뷰는 부정 입니다.


리뷰데이터 : 안좋다
>> 입력한 리뷰는 부정 입니다.


